# Curate_StudyAppSatisfaction_Data

In [1]:
%matplotlib inline

import datetime as dt
import itertools as it

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import pandas as pd
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

tqdm.pandas()

Welcome, Abhishek Pratap!



In [3]:
v1sid = 'syn10250484'
v1r = pd.read_excel(syn.get(v1sid).path, parse_dates=['response_utc'])

In [ ]:
v1 = v1r.drop(columns=[
    'start', 'user_id', 'sent_time_local', 'sent_time_utc', 'response_local', 'response_id'
]).rename(columns={
    'brightenid':'participant_id',
    'response_utc':'dt_response',
    'What did you hope your assigned study app (Evolution, Problem Solving Therapy, or Ginger.io) would do for you?':'sat_1',
    'Did the assigned study app meet your goals?':'sat_2',
    'Do you plan to continue using the assigned study app?':'sat_3',
    'Would you recommend this assigned study app to others?':'sat_4'
}).loc[:, ['participant_id', 'dt_response', 'week', 'sat_1', 'sat_2', 'sat_3', 'sat_4']]


def fx(x):
    if x == 'Yes':
        return 1
    elif x == 'No':
        return 0
    else:
        raise ValueError(f'mapping not defined: {x}')
        
v1.sat_2 = v1.sat_2.apply(fx)
v1.sat_3 = v1.sat_3.apply(fx)
v1.sat_4 = v1.sat_4.apply(fx)

def fx(x):
    if isinstance(x, str):
        return x
    else:
        return '<no response>'

v1.sat_1 = v1.sat_1.apply(fx)
    
# localize timestamps
v1['dt_response'] = [
    str(t.tz_localize('UTC'))
    for t in v1.dt_response
]

v1.head()

### Recalculate the week into study

In [ ]:
# metadata = syn.tableQuery('SELECT participant_id, startdate FROM syn17023349').asDataFrame(convert_to_datetime=True)
# metadata.startdate = pd.to_datetime(metadata.startdate)

# v1 = pd.merge(v1, metadata, on='participant_id', how='left')
# v1['week'] = [d.days/7 for d in (v1.dt_response-v1.startdate)]
# v1['week'] = v1.week.progress_apply(lambda x: np.int16(np.floor(x)) if not pd.isnull(x) else np.nan)
# v1 = v1.drop(columns=['startdate'])

# v1.head()

### Upload to Synapse

In [ ]:
# check for dupes in original
t = syn.delete(
    syn.tableQuery('SELECT * FROM syn17025202')
)

In [ ]:
final = syn.store(Table(
    Schema(
            name='Application Satisfaction',
            columns=as_table_columns(v1), 
            parent='syn10848316'),
        v1
    )
)

In [4]:
final = syn.setProvenance(
    'syn17025202',
    activity=Activity(
        name='Combine V1 and V2 data',
        description='Process and combine the data collected during study 1 and study 2',
        used=[v1sid],
        executed=[
            dict(
                name='Curate_StudyAppSatisfaction_Data',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Curate_StudyAppSatisfaction_Data.ipynb'
            )
        ]
    )
)